通常我们在处理图像时，我们希望逐渐降低隐藏表示的空间分辨率、聚集信息，
这样随着我们在神经网络中层叠的上升，每个神经元对其敏感的感受野输入就越大。

而我们的机器学习任务通常会跟全局图像的问题相关。（例如，“图像是否包含一只猫呢？”）

##### 所以我们最后一层的神经元应该对整个输入的全局敏感
通过逐渐聚合信息，生成越来越粗糙的映射，最终实现学习全局表示的目标，同时将卷积图层的所有优势保留在中间层。
此外，当检测较底层的特征时（例如 6.2节中所讨论的边缘），我们通常希望这些特征保持某种程度上的平移不变性。例如，如果我们拍摄黑白之间轮廓清晰的图像X，并将整个图像向右移动一个像素，即Z[i, j] = X[i, j + 1]，则新图像Z的输出可能大不相同。而在现实中，随着拍摄角度的移动，任何物体几乎不可能发生在同一像素上。即使用三脚架拍摄一个静止的物体，由于快门的移动而引起的相机振动，可能会使所有物体左右移动一个像素（除了高端相机配备了特殊功能来解决这个问题）。

本节将介绍汇聚（pooling）层，它具有双重目的：
+ 降低卷积层对位置的敏感性
+ 降低对空间降采样表示的敏感性

### 6.5.1. 最大汇聚层和平均汇聚层
在下面的代码中的pool2d函数，我们实现汇聚层的前向传播。 此功能类似于 6.2节中的corr2d函数。 然而，这里我们没有卷积核，输出为输入中每个区域的最大值或平均值。

In [1]:
import torch
from torch import nn
from d2l import torch as d2l

In [3]:
def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i : i + p_h, j : j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i : i + p_h, j : j + p_w].mean()
    return Y

In [4]:
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
X

tensor([[0., 1., 2.],
        [3., 4., 5.],
        [6., 7., 8.]])

In [6]:
pool2d(X, pool_size=(2,2), mode='avg')

tensor([[2., 3.],
        [5., 6.]])

In [7]:
pool2d(X, pool_size=(2,2), mode='max')

tensor([[4., 5.],
        [7., 8.]])

### 6.5.2. 填充和步幅
与卷积层一样，汇聚层也可以改变输出形状。和以前一样，我们可以通过填充和步幅以获得所需的输出形状。 下面，我们用深度学习框架中内置的二维最大汇聚层，来演示汇聚层中填充和步幅的使用。 我们首先构造了一个输入张量X，它有四个维度，其中样本数和通道数都是1。

In [8]:
X = torch.arange(16, dtype=torch.float32).reshape((1,1,4,4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

默认情况下，深度学习框架中的
##### 步幅与汇聚窗口pool的大小相同，也就是说，如果我们使用形状为(3, 3)的pool，那么步幅也是(3, 3)

In [14]:
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

填充和步幅可以手动设定。

In [15]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

当然，我们可以设定一个任意大小的矩形汇聚窗口，并分别设定填充和步幅的高度和宽度。

In [16]:
pool2d = nn.MaxPool2d((2, 3), stride=(2, 3), padding=(0, 1))
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

### 6.5.3. 多个通道
在处理多通道输入数据时，汇聚层再每个输入通道上单独运算。
而不是像卷积一样再通道上对输入汇总。
这意味着汇聚成的输出通道数与输入通道数相同

In [19]:
X, X + 1

(tensor([[[[ 0.,  1.,  2.,  3.],
           [ 4.,  5.,  6.,  7.],
           [ 8.,  9., 10., 11.],
           [12., 13., 14., 15.]],
 
          [[ 1.,  2.,  3.,  4.],
           [ 5.,  6.,  7.,  8.],
           [ 9., 10., 11., 12.],
           [13., 14., 15., 16.]]]]),
 tensor([[[[ 1.,  2.,  3.,  4.],
           [ 5.,  6.,  7.,  8.],
           [ 9., 10., 11., 12.],
           [13., 14., 15., 16.]],
 
          [[ 2.,  3.,  4.,  5.],
           [ 6.,  7.,  8.,  9.],
           [10., 11., 12., 13.],
           [14., 15., 16., 17.]]]]))

In [23]:
X = torch.cat((X, X+1),1)
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]],

         [[ 2.,  3.,  4.,  5.],
          [ 6.,  7.,  8.,  9.],
          [10., 11., 12., 13.],
          [14., 15., 16., 17.]],

         [[ 1.,  2.,  3.,  4.],
          [ 5.,  6.,  7.,  8.],
          [ 9., 10., 11., 12.],
          [13., 14., 15., 16.]],

         [[ 2.,  3.,  4.,  5.],
          [ 6.,  7.,  8.,  9.],
          [10., 11., 12., 13.],
          [14., 15., 16., 17.]],

         [[ 2.,  3.,  4.,  5.],
          [ 6.,  7.,  8.,  9.],
          [10., 11., 12., 13.],
          [14., 15., 16., 17.]],

         [[ 3.,  4.,  5.,  6.],
          [ 7.,  8.,  9., 10.],
          [11., 12., 13., 

如下所示，汇聚后输出通道的数量仍然是2。

In [24]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]],

         [[ 6.,  8.],
          [14., 16.]],

         [[ 6.,  8.],
          [14., 16.]],

         [[ 7.,  9.],
          [15., 17.]],

         [[ 6.,  8.],
          [14., 16.]],

         [[ 7.,  9.],
          [15., 17.]],

         [[ 7.,  9.],
          [15., 17.]],

         [[ 8., 10.],
          [16., 18.]]],


        [[[ 6.,  8.],
          [14., 16.]],

         [[ 7.,  9.],
          [15., 17.]],

         [[ 7.,  9.],
          [15., 17.]],

         [[ 8., 10.],
          [16., 18.]],

         [[ 7.,  9.],
          [15., 17.]],

         [[ 8., 10.],
          [16., 18.]],

         [[ 8., 10.],
          [16., 18.]],

         [[ 9., 11.],
          [17., 19.]]],


        [[[ 6.,  8.],
          [14., 16.]],

         [[ 7.,  9.],
          [15., 17.]],

         [[ 7.,  9.],
          [15., 17.]],

         [[ 8., 10.],
          [16., 18.]],

         [[ 7.,  9.],
          [15., 17.]],

         [[ 8., 10.],
        

### 6.5.4. 小结
...
...
使用汇聚层以及大于1的步幅，可以减少 ***空间维度（如高度与宽度）***
...